# Create Performance Task WriteUp
> Sample Writeup of Create Performance Task

- toc: true
- categories: []
- image: /images/apcsp.png

## Row 1: Program Purpose and Function
### Video
> The video demonstrates the running of the program.  You must show...

- input
- program functionality
- output


### Wrtten Response
- Describes the overall purpose of the program.
    - The purpose of this program is to add and display users and their identity information.  
    - Most Web Applications require user accounts and user information.  This program is intended to be the basis of building a future system with users and roles based off of their identity information.


- Describes what functionality of the program is demonstrated in the video.
    - Demonstrated in the video is a JavaScript/HTML/CSS program.  This program is used to Create and Read users.
    - Users are displayed when the page is loaded.  The listing, which is in table format, contains common identity information required for a user/account system.  
    - The information in the page is actually read from a different web application.  These users are retrieved from a backend system, an entirely different server and web application.  The users are stored in an SQLite Database and retrieved from this other web application via a backend Web API and endpoint, using a GET method
    - Additionally, in the video shows the process for adding/creating new user information, this is with an API and enpoint use a POST Method.   After the POST, the information input on screen is validated and updated in the backend SQLite database, also it is updated on frontend Web page within the


- Describes the input and output of the program demonstrated in the video.
    - Initially, the JavaScript obtains input from the Web GET API, the response from the API contains JSON.   This JSON file is a collection containing a list of dictionaries which is iterated over in JavaScript row by row, each row outputs a corresponding row on the Web Page by generating HTML table data containing each user and their columnar data. 
    - Additionally, in the demonstration of creating new user information a WEB POST API enpoint is called.  In this case, the data is passed in the request body.  The backend API validates and unpacks the JSON data into a Python Object.  The Object contains the ability to Create a new row in the SQLite table.  After data is validated and created, it return success response to JavaScript, at that time the data input is displayed in the HTML table, add a row and columnar data for the new user.

## Row 2: Data Abstraction
> Below are some key ideas to better illustrate multiple levels of Data Abstraction within this project.  The frontend allows for display, the backend API allows for JSON transport over the internet, and the backend database and inheritance handle interaction with SQL.

- includes two program code segments: 
    - one that shows how data has been stored in this list (or other collection type).
        - The majority of the code below is in a Python file called users.py, the imports are included to avoid warnings in the Jupyter notebook.
        - The collection type used is an SQLite database, the class User(db.Model) inherits database functionality.
        - Below the Class definition there is an initUsers() function.  This function create a database table and shows how to create a User objects in Python and then adds each of the User objects to a database table called 'users'. This creates and tests the process of creating records in the backend.  Such a function is used to allow testing of database without the frontend or the API. 
        - Each User needs goes through the process of validation and checking through try/except without impacting future operations, meaning if Thomas Edison failed then remaining users will still run and be added if validation passes. 
        - The user.create() method in the class creates a new row in the database table.  Additionally, it validates database operation, specifically it makes sure uid is unique, no duplicates.
        - The visual is a screen capture of the user table after this code is run.

In [ ]:
from datetime import date
import json

from sqlalchemy.exc import IntegrityError
from werkzeug.security import generate_password_hash, check_password_hash

from flask import Flask
from flask_sqlalchemy import SQLAlchemy

app = Flask(__name__)
# Setup SQLAlchemy object and properties for the database (db)
dbURI = 'sqlite:///volumes/sqlite.db'
app.config['SQLALCHEMY_TRACK_MODIFICATIONS'] = False
app.config['SQLALCHEMY_DATABASE_URI'] = dbURI
app.config['SECRET_KEY'] = 'SECRET_KEY'
db = SQLAlchemy()

class User(db.Model):
    __tablename__ = 'users'  # table name is plural, class name is singular

    # Define the User schema with "vars" from object
    id = db.Column(db.Integer, primary_key=True)
    _name = db.Column(db.String(255), unique=False, nullable=False)
    _uid = db.Column(db.String(255), unique=True, nullable=False)
    _password = db.Column(db.String(255), unique=False, nullable=False)
    _dob = db.Column(db.Date)

    # Defines a relationship between User record and Notes table, one-to-many (one user to many notes)
    posts = db.relationship("Post", cascade='all, delete', backref='users', lazy=True)

    # constructor of a User object, initializes the instance variables within object (self)
    def __init__(self, name, uid, password="123qwerty", dob=date.today()):
        self._name = name    # variables with self prefix become part of the object, 
        self._uid = uid
        self.set_password(password)
        self._dob = dob

    # a name getter method, extracts name from object
    @property
    def name(self):
        return self._name
    
    # a setter function, allows name to be updated after initial object creation
    @name.setter
    def name(self, name):
        self._name = name
    
    # a getter method, extracts email from object
    @property
    def uid(self):
        return self._uid
    
    # a setter function, allows name to be updated after initial object creation
    @uid.setter
    def uid(self, uid):
        self._uid = uid
        
    # check if uid parameter matches user id in object, return boolean
    def is_uid(self, uid):
        return self._uid == uid
    
    @property
    def password(self):
        return self._password[0:10] + "..." # because of security only show 1st characters

    # update password, this is conventional setter
    def set_password(self, password):
        """Create a hashed password."""
        self._password = generate_password_hash(password, method='sha256')

    # check password parameter versus stored/encrypted password
    def is_password(self, password):
        """Check against hashed password."""
        result = check_password_hash(self._password, password)
        return result
    
    # dob property is returned as string, to avoid unfriendly outcomes
    @property
    def dob(self):
        dob_string = self._dob.strftime('%m-%d-%Y')
        return dob_string
    
    # dob should be have verification for type date
    @dob.setter
    def dob(self, dob):
        self._dob = dob
    
    @property
    def age(self):
        today = date.today()
        return today.year - self._dob.year - ((today.month, today.day) < (self._dob.month, self._dob.day))
    
    # output content using str(object) in human readable form, uses getter
    # output content using json dumps, this is ready for API response
    def __str__(self):
        return json.dumps(self.read())

    # CRUD create/add a new record to the table
    # returns self or None on error
    def create(self):
        try:
            # creates a person object from User(db.Model) class, passes initializers
            db.session.add(self)  # add prepares to persist person object to Users table
            db.session.commit()  # SqlAlchemy "unit of work pattern" requires a manual commit
            return self
        except IntegrityError:
            db.session.remove()
            return None

    # CRUD read converts self to dictionary
    # returns dictionary
    def read(self):
        return {
            "id": self.id,
            "name": self.name,
            "uid": self.uid,
            "dob": self.dob,
            "age": self.age,
            "posts": [post.read() for post in self.posts]
        }
        
# Builds working data for testing
def initUsers():   
    with app.app_context():
        """Create database and tables"""
        db.create_all()
        """Tester data for table"""
        u1 = User(name='Thomas Edison', uid='toby', password='123toby', dob=date(1847, 2, 11))
        u2 = User(name='Nicholas Tesla', uid='niko', password='123niko')
        u3 = User(name='Alexander Graham Bell', uid='lex', password='123lex')
        u4 = User(name='Eli Whitney', uid='whit', password='123whit')

        users = [u1, u2, u3, u4]

        """Builds sample user/note(s) data"""
        for user in users:
            try:
                user.create()
            except IntegrityError:
                '''fails with bad or duplicate data'''
                db.session.remove()
                print(f"Records exist, duplicate email, or error: {user.uid}")
        
        

![](images/userstable.png)

- one that shows the data in this same list being used as part of fulfilling the program’s purpose.
    - The Frontend obtains an API response, inside the response is a Collection called JSON.  See response.json().then(data => to see how the JSON is obtained from API resonse into the "data" variable.
    - The JSON response was formed in the backend, this data was abstracted from an SQLite database.
    - The JavaScript syntax, via labda expression, allws the "data" to be abstracted into a JavaScript variable.
    - The data is passed as a parameter to the add_row function.
    
- identifies the name of the variable representing the list being used in this response.
    - the variable is called "data"
    
- describes what the data contained in this list is representing in the program.
    - JavaScript supports a data.key notation, this allows abstraction of the data from this key which is  assigned into innerHTML on the page and in the table.  
    - Keys are uid, name, dob, and age, the values of these keys are assigned to the innerHTML.

In [ ]:
// response contains valid result
response.json().then(data => {
    //add a table row for the new/created userid
    add_row(data);
})

In [ ]:
// HTML table row and columnar data generation
function add_row(data) {
    const tr = document.createElement("tr");
    const uid = document.createElement("td");
    const name = document.createElement("td");
    const dob = document.createElement("td");
    const age = document.createElement("td");
  

    // obtain data that is specific to the API
    uid.innerHTML = data.uid; 
    name.innerHTML = data.name; 
    dob.innerHTML = data.dob; 
    age.innerHTML = data.age; 

    // add HTML to container
    tr.appendChild(uid);
    tr.appendChild(name);
    tr.appendChild(dob);
    tr.appendChild(age);

    resultContainer.appendChild(tr);
  }

![](images/usershtml.png)

- Backend, obtaining from Database List or Collection using Python Database Class.  In my Python API preparation code which returns a response to frontend…   This is reply from a Database query.  The collection is called “users”.

In [ ]:
def get(self):
    users = User.query.all()    # read/extract all users from database
    json_ready = [user.read() for user in users]  # prepare output in json 
    return jsonify(json_ready)  # jsonify creates Flask response object,  specific to APIs with response and dat

## ROW 3.  Managing Complexity.  
> Data creation and storage is used to illustrate "managing complexity".  Since input is coming from users, data needs to be validate prior to adding it to the system.  By creating a User object, data can be stagged and validated utilizing methods that are defined in the collection/object discussed in this section.

- includes a program code segment that shows a list being used to manage complexity in the program.
    - The ```class _Create(Resource):``` has method ```def post(self):``` contains the variable ```body```.  This is populated with the user input.  This collection contains a list of all the keys and values input by user in one varialble.  
        - the variable ```body``` contains the user input including: uid, name, dob
    - The input data after it passes initial checks for errors is used to create an object ```uo = User(name=name, uid=uid)```.   The uo object, user object, is assisting in managing the complexity of the user data.  The method ```def post(self)``` checks data and add it to the uo object as it ensures only clean data enters the database.  Ei

- explains how the named, selected list manages complexity in the program code by explaining why the program code could not be written, or how it would be written differently, without using this list.
    - The "uo" object, after it is created contains data from the user input, as well as a method to Create, ```uo.create()`` a record in the table.  The object activates code, through this method, that does not need to be throroughly understood by the implementor of the API.
    - Additionally, there is a method to set the password, ```uo.set_password(password)```.  This method abstracts complexity of password encryption.
    - Also, there is a method to set the date of birth, ```uo.dob = datetime.strptime(dob, '%Y-%m-%d').date()```.  Behind the scenes this calculates age of the user.
    - Ultimately, after object is generated and all data is verified, the ```uo.create()``` method is called and a new row is committed into the database table.  In this step, all the commands required to interact with the SQL database have been abstracted.
    - There are ways to write the ```class User``` code differently!  Primarly to make the code into a imperative sequence and perform everything done within the object in a serialized form of commands.  However, this would minimize the possibilities for code reuse.  For instance, in a ***User Management System*** there would be a need to ***Update*** user records, as well as this mentioned ***Create*** user record.   The user class and creating a ```uo``` object would similarly contain methods to support ***Update*** functionality.  For instance, the same  ```password``` and ```dob``` mutator/setter methods work for update.  Thus, an update API would be able to use these abstracted methods without a code modification or rewrite.   
    - The concept of passing a list of Key/Value pairs through JSON, as illustrated with ```body``` variable is a foundation piece for REST API development.
    - The concept of creating/updating a user object (```uo```) is common to Object Oriented Programming (OOP).  This enables the management of an Object (ie user object), which is a series of data elements, along with the code to  mutate and access that data in a reusable and reliable manner.  All things in code can be done in alternate ways, but OOP is great way to manage complexity on a list of attributes describing an object, aka user.

In [ ]:
from flask import Blueprint, request, jsonify
from flask_restful import Api, Resource # used for REST API building
from datetime import datetime

user_api = Blueprint('user_api', __name__,
                   url_prefix='/api/users')

# API docs https://flask-restful.readthedocs.io/en/latest/api.html
api = Api(user_api)

class UserAPI:        
    class _Create(Resource):
        def post(self):
            ''' Read data for json body '''
            body = request.get_json()
            
            ''' Avoid garbage in, error checking '''
            # validate name
            name = body.get('name')
            if name is None or len(name) < 2:
                return {'message': f'Name is missing, or is less than 2 characters'}, 400
            # validate uid
            uid = body.get('uid')
            if uid is None or len(uid) < 2:
                return {'message': f'User ID is missing, or is less than 2 characters'}, 400
            # look for password and dob
            password = body.get('password')
            dob = body.get('dob')

            ''' #1: Key code block, setup USER OBJECT '''
            uo = User(name=name, 
                      uid=uid)
            
            ''' Additional garbage error checking '''
            # set password if provided
            if password is not None:
                uo.set_password(password)
            # convert to date type
            if dob is not None:
                try:
                    uo.dob = datetime.strptime(dob, '%Y-%m-%d').date()
                except:
                    return {'message': f'Date of birth format error {dob}, must be mm-dd-yyyy'}, 400
            
            ''' #2: Key Code block to add user to database '''
            # create user in database
            user = uo.create()
            # success returns json of user
            if user:
                return jsonify(user.read())
            # failure returns error
            return {'message': f'Processed {name}, either a format error or User ID {uid} is duplicate'}, 400


## ROW 4. Procedural Abstraction  
> In a user system there is a need to store a password.  However, it is not appropriate to store a password without encryption.  Answer to questions and code snippets follow.

- includes two program code segments:
    - one showing a student-developed procedure with at least one parameter that has an effect on the functionality of the procedure.
        - the code fragment below shows a procedure that sets the password in the user object ```self._password```.  It receive a parameter ```password``` from the caller of the method.
        - the password is hashed before it is saved in the object using ```generate_password_hash``` method. this method is part of the ***werkzeug.security*** library.  
        - a ```password``` is required to create a ***sha256 hashed*** password as a result

    - one showing where the student-developed procedure is being called.
        - the ```uo.set_password(password)``` below is prefaced with the dependencies before the call.   The password is one attribute in a larger ***user object***
        - the ```password``` is obtained from input
        - this input ```password``` is passed as a parameter to the user object set_password method as shown

- describes what the identified procedure does and how it contributes to the overall functionality of the program.
    - a ***User System*** requires a passowrd to validate identity of the user
    - the set_password method is focus of this answer, but to support need a second method ```def is_password(self, password):``` is added below.   This method is used actually ***validate identity***.  This method has a dependency that the hashed password is stored, it compares the passowrd passed as a paramter versus the hashed password in the object (```self._password```)
    - together to ***set*** and ***is*** methods can be used to manage passwords and validate identy of the users

In [ ]:
# update password, this is conventional setter
def set_password(self, password):
    """Create a hashed password."""
    self._password = generate_password_hash(password, method='sha256')

# check password parameter versus stored/encrypted password
def is_password(self, password):
    """Check against hashed password."""
    result = check_password_hash(self._password, password)
    return result

In [ ]:
from flask import request

# Dependencies for object creation
body = request.get_json()
name = body.get('name')
uid = body.get('uid')
uo = User(name=name, uid=uid)   
# Dependencies for obtaining password from user input
password = body.get('password')
            
''' Call to set_password method '''
uo.set_password(password)

## ROW 5. Algorithm Implementation 
> The most highly visible algorithm in this program is the code that displays the users.  This contains all the elements required: sequencing, selection, iteration.

- includes a program code segment of a studentdeveloped algorithm that includes
    - sequencing
    - selection
    - iteration

- explains in detailed steps how the identified algorithm works in enough detail that someone else could recreate it.
    - the function contain the algorithm is called ```read_users()``` which has the purpose to display all Users in the database.
    - in JavaScript, the fetch() method is used to make a request to the server, this begins the sequence
        -  ```var url = "https://flask.nighthawkcodingsociety.com/api/users"``` is the endpoint for API
        - ```method: GET``` means that we intend to retrieve information
        - ```mode: cors``` states that API is cross origin
        - ```Content-Type: application/json``` describes how we are passing data
    - in JavaScript, the response of the fetch() using GET method contains status and data, this continues the sequence and will enable need for both select and iteration
        - ```response.status``` contains valid respone 200, or all other respones.
        - the selection statement ```if (response.status !== 200) {``` traps all normal error conditions and builds an HTML container to display those errors to user.  Normal errors are handled by endpoint, such as invalid date.
        - the ```.catch``` at bottom of procedure is another type of selection statement that traps all errors that are unexpected.  This would be errors that do not reach the endpoint, such as server being down.
        - passing by the guards allows the sequence of code to progress to ```response.json().then(data => {```.  This is a lamda expression that extracts the data and enables JavaScript to load the information that is returned by the server onto the web pages. 
        - the iteration statement ```for (let row in data) {``` extracts each row from the data.  The row is JSON and work as key/value pairs.
        - through each iteration a call to ```add_row(data[row]);``` loads the data on screen

In [ ]:
// Display User Table, data is fetched from Backend Database
var url = "https://flask.nighthawkcodingsociety.com/api/users"

function read_users() {
  // prepare fetch options
  const read_options = {
    method: 'GET', // *GET, POST, PUT, DELETE, etc.
    mode: 'cors', // no-cors, *cors, same-origin
    cache: 'default', // *default, no-cache, reload, force-cache, only-if-cached
    credentials: 'omit', // include, *same-origin, omit
    headers: {
      'Content-Type': 'application/json'
    },
  };

  // fetch the data from API
  fetch(read_fetch, read_options)
    // response is a RESTful "promise" on any successful fetch
    .then(response => {
      // check for response errors
      if (response.status !== 200) {
          const errorMsg = 'Database read error: ' + response.status;
          console.log(errorMsg);
          const tr = document.createElement("tr");
          const td = document.createElement("td");
          td.innerHTML = errorMsg;
          tr.appendChild(td);
          resultContainer.appendChild(tr);
          return;
      }
      // valid response will have json data
      response.json().then(data => {
          console.log(data);
          for (let row in data) {
            console.log(data[row]);
            add_row(data[row]);
          }
      })
  })
  // catch fetch errors (ie ACCESS to server blocked)
  .catch(err => {
    console.error(err);
    const tr = document.createElement("tr");
    const td = document.createElement("td");
    td.innerHTML = err;
    tr.appendChild(td);
    resultContainer.appendChild(tr);
  });
}

## ROW 6. Testing
> The frontend makes two HTTP method calls (post, get) to the backend API endpoint.  A single API endpoint is defined in the ```clase UserAPI``` it is at the bottom of code sequence ```api.add_resource(_CRUD, '/')```.  The supported backend methods ```post(self), get(self)``` can be found inside of the ```class _CRUD(Resource): ```.

- describe two calls to the selected procedure identified in written response 3c. Each call must pass a different argument(s) that causes a different segment of code in the algorithm to execute.
    - The backend API ```class _CRUD(Resource):``` detects the GET or POST method through the ```Resource``` and directs the action to the ```def get(self):``` or the ```def post(self):``` method.

    
- describes the condition(s) being tested by each call to the procedure.
-    The Two methods test the Create, Read database actions.  
    - The first, API POST method, ```def post(self):```. Creates a new row in the backend database.
    - The second, API GET method, ```def get(self):```.  Reads data from the backend database.
    
- identifies the result of each call.
    - The first, creates a record and performs ```return jsonify(user.read())``` which contains the newly create user row.  Alternately, it could return one or more errors ```return {'message': f'User ID is missing, or is less than 2 characters'}, 400``` based off of invalid user request.  On succesfull return the data is loaded and displayed by the JavaScript Web server
    - The second, produces a list of users from a database query ```users = User.query.all()```, turns the query into a Python list, using list comprenhension ```json_ready = [user.read() for user in users]```.  Then it returns a Status code and on success returns JSON data ```return jsonify(json_ready)```.  On return the data is loaded and displayed by the JavaScript Web server

In [ ]:
from flask import Blueprint, request, jsonify
from flask_restful import Api, Resource # used for REST API building
from datetime import datetime

user_api = Blueprint('user_api', __name__,
                   url_prefix='/api/users')

# API docs https://flask-restful.readthedocs.io/en/latest/api.html
api = Api(user_api)

class UserAPI:        
    class _CRUD(Resource):  # this class exposes supported HTTP methods (post, get, etc)
        def post(self):
            ''' Read data for json body '''
            body = request.get_json()
            
            ''' Avoid garbage in, error checking '''
            # validate name
            name = body.get('name')
            if name is None or len(name) < 2:
                return {'message': f'Name is missing, or is less than 2 characters'}, 400
            # validate uid
            uid = body.get('uid')
            if uid is None or len(uid) < 2:
                return {'message': f'User ID is missing, or is less than 2 characters'}, 400
            # look for password and dob
            password = body.get('password')
            dob = body.get('dob')

            ''' #1: Key code block, setup USER OBJECT '''
            uo = User(name=name, 
                      uid=uid)
            
            ''' Additional garbage error checking '''
            # set password if provided
            if password is not None:
                uo.set_password(password)
            # convert to date type
            if dob is not None:
                try:
                    uo.dob = datetime.strptime(dob, '%Y-%m-%d').date()
                except:
                    return {'message': f'Date of birth format error {dob}, must be mm-dd-yyyy'}, 400
            
            ''' #2: Key Code block to add user to database '''
            # create user in database
            user = uo.create()
            # success returns json of user
            if user:
                return jsonify(user.read())
            # failure returns error
            return {'message': f'Processed {name}, either a format error or User ID {uid} is duplicate'}, 400

        def get(self):
            users = User.query.all()    # read/extract all users from database
            json_ready = [user.read() for user in users]  # prepare output in json
            return jsonify(json_ready)  # jsonify creates Flask response object, more specific to APIs than json.dumps


    # UserAPI RESTful API endpoint
    api.add_resource(_CRUD, '/')    